In [16]:
%run "Import.ipynb"

class DataAnalysis(DataOperation):
    def analyze_and_validate(self):
        """
        Führt die Analyse der Trainingsdaten durch. 
        1.Findet die besten Passungen mit den idealen Funktionen.
        2.Berechnung der maximalen Abweichungen zwischen Trainingsdaten und den idealen Funktionen
        3.Validiert die Testdaten gegen die Passungen. Speichert die Ergebnisse in der Datenbank.

        Args:
        Keine

        Raises:
        Exception: Gibt einen Fehler aus, wenn die SQL-Operationen fehlschlagen.
        Exception: Gibt einen Fehler aus, wenn die Speicherung der validierten Testergebnisse fehlschlägt.

        Returns:
        best_fits_df (Dataframe), max_deviations (Dict.), valid_test_results_df (Dataframe)
        """
        # Laden Trainings-, Test und ideale Daten
        try:
            train_df = pd.read_sql('SELECT * FROM training_data', self.conn)
            ideal_df = pd.read_sql('SELECT * FROM ideal_functions', self.conn)
            test_df = pd.read_sql('SELECT * FROM test_data', self.conn)
        except Exception as e:
            print(f"Fehler beim Laden der Daten: {e}")
            raise
        finally:
            print("Daten erfolgreich aus SQL übernommen")
            
        # Findet die besten Passungen für die Trainingsdaten anhand der least-Square Methode
        best_fits = []
        try:
            for i in range(1, 5):  # Für y1 bis y4 in den Trainingsdaten
                best_fit, lowest_sum_sq_diff = None, np.inf
                for col in ideal_df.columns[1:]:  # ohne x-Spalte
                    sq_diff = np.sum((train_df[f'y{i}'] - ideal_df[col]) ** 2)
                    if sq_diff < lowest_sum_sq_diff:
                        best_fit, lowest_sum_sq_diff = col, sq_diff
                best_fits.append((f'y{i}', best_fit, lowest_sum_sq_diff))
                best_fits_df = pd.DataFrame(best_fits, columns=["Traingsdatensatz", "Beste Passung", "Summe aller quadratischen y-Abweichungen"])
        except Exception as e:
            print(f"Fehler beim Finden der besten Passungen: {e}")
        finally:
            print("Best Fits gefunden")

        # Berechnung der maximalen Abweichungen zwischen Trainingsdaten und den idealen Funktionen
        max_deviations = {}
        try:
            for train_y, best_fit, _ in best_fits:
                # Berechnung der maximale Abweichung zwischen der Trainingsdaten-y-Spalte und der besten idealen Funktion
                max_deviation = np.max(np.abs(train_df[train_y] - ideal_df[best_fit]))
                max_deviations[best_fit] = max_deviation
        except Exception as e:
            print(f"Fehler bei der Berechnung der max. Abweichung: {e}")
        finally:
            print("Max. Abweichungen gefunden")
            
        # Validierung der Testdaten gegen die besten Passungen
        valid_test_results = []
        max_allowed_deviation_factor = np.sqrt(2)
        try:
            for _, row in test_df.iterrows():
                x_value, y_test = row['x'], row['y']
                for ideal_col, max_dev in max_deviations.items():
                    y_ideal = np.interp(x_value, ideal_df['x'], ideal_df[ideal_col])
                    deviation = np.abs(y_test - y_ideal)
                    if deviation <= max_dev * max_allowed_deviation_factor:
                        valid_test_results.append((x_value, y_test, ideal_col, deviation))
                        break  # Stoppt die Schleife, wenn eine passende ideale Funktion gefunden wurde
        except Exception as e:
            print(f"Fehler bei der Validierung: {e}")
        finally:
            print("Validierte Testdaten erstellt")

        # Speichert die validierten Testergebnisse in der Datenbank
        try:
            valid_test_results_df = pd.DataFrame(valid_test_results, columns=['x', 'y', 'Ideale Funktion', 'Abweichung'])
            valid_test_results_df.to_sql('validated_test_results', self.conn, if_exists='replace', index=False)
        except Exception as e:
            print(f"Fehler beim speichern der validierten Testergebnisse: {e}")
            raise
        finally:
            print("Testdaten validiert und in 'validated_test_results' gespeichert.")
        # Validierte Testdaten anzeigen
        return best_fits_df, max_deviations, valid_test_results_df

# Initialisierung und Ausführung
data_analysis = DataAnalysis('sqldatabase.sqlite')
data_analysis.connect_to_db()
data_analysis.analyze_and_validate()

DB-Verbindung i.O.
SQL-DB erstellt
Daten erfolgreich importiert zu training_data.
Daten erfolgreich importiert zu ideal_functions.
Daten erfolgreich importiert zu test_data.
SQL-Tabellen erstellt
DB-Verbindung i.O.
Daten erfolgreich aus SQL übernommen
Best Fits gefunden
Max. Abweichungen gefunden
Validierte Testdaten erstellt
Testdaten validiert und in 'validated_test_results' gespeichert.


(  Traingsdatensatz Beste Passung  Summe aller quadratischen y-Abweichungen
 0               y1           y36                                 33.711789
 1               y2           y11                                 32.628931
 2               y3            y2                                 33.118472
 3               y4           y33                                 31.752431,
 {'y36': 0.4997939999999943,
  'y11': 0.49678800000000045,
  'y2': 0.49987616,
  'y33': 0.4949089999999998},
        x          y Ideale Funktion  Abweichung
 0    4.9   4.496336             y11    0.403664
 1    7.2   6.698579             y11    0.501421
 2   17.9  17.754583             y11    0.145417
 3   -5.6   0.995935              y2    0.220370
 4   19.7  19.575151             y11    0.124849
 5   -2.9  13.796734             y36    0.703266
 6    4.7   4.568574             y11    0.131426
 7   14.5  14.734339             y11    0.234339
 8   -2.8  13.655836             y36    0.344164
 9    6.9   7.529588